In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install nltk

In [3]:
!pip install pyvi

     |████████████████████████████████| 8.5MB 5.7MB/s 
     |████████████████████████████████| 747kB 33.3MB/s 


In [0]:
!pip install keras

In [0]:
#import and load data
import nltk
from nltk.stem import WordNetLemmatizer
lemmatize= WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

#be ready to load data
words       =[] # mảng lưu trũ các từ sau khi được tách ra khỏi câu
classes     =[] # mảng lưu trữ các tag
documents   =[] # tập hợp các từ tương ứng với tag của chứng cho mỗi intent

with open('/content/drive/My Drive/Python code/Chatbot/weather bot/stopwords.txt') as f:
    ignore_words=f.read().splitlines()

# print(type(ignore_words))
# print(ignore_words)
#load data
data_file   = open('/content/drive/My Drive/Python code/Chatbot/weather bot/weather_intents.json').read()
intents     = json.loads(data_file)

In [0]:
# data_file   = open('/content/drive/My Drive/Python code/Chatbot/intents2.json').read()
# print(data_file)

In [0]:
# data_file   = open('/content/drive/My Drive/Python code/Chatbot/intents.json').read()
# print(data_file)

In [26]:
from pyvi import ViTokenizer
nltk.download('punkt')
#preprocessing data
for intent in intents["intents"]:
  for pattern in intent["patterns"]:

    #tokenize each word
    # w= nltk.word_tokenize(pattern)
    w= ViTokenizer.tokenize(pattern)
    words.extend(w) # Thêm các từ vào tập các từ
    #add document to corpus
    documents.append((w,intent['tag']))

    #add to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
nltk.download('wordnet')
#lemmatize: chuyển hóa các từ trong tiếng anh về dạng nguyên thể của nó.
# words   = [lemmatize.lemmatize(w.lower()) for w in words if w not in ignore_words]
words   = [w.lower() for w in words if w not in ignore_words]
words   = sorted(list(set(words))) #delete the duplicate in words
classes = sorted(list(set(classes)))

#document is combination of pattern and intents
print("there are", len(documents),"documents")
print("All class")
for i, class_ in enumerate(classes):
  print("%d : %s"%(i,class_))
print("number of unique words: ", len(words))

pickle.dump(words,open("/content/drive/My Drive/Python code/Chatbot/weather bot/weather_words2.pkl","wb"))
pickle.dump(classes,open("/content/drive/My Drive/Python code/Chatbot/weather bot/weather_classes2.pkl","wb"))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
there are 192 documents
All class
0 : goodbye
1 : greeting
2 : precipitation
3 : sunny
4 : temperature
5 : thanks
6 : time
7 : uv
8 : weather
number of unique words:  61


In [29]:
#create training and testing data

#create trainning data
training=[]
#create an empty array for output
output_array= [0]*len(classes)
# training set, bag of words for each sentence
for doc in documents:
  #initial bag of words
  bag=[]
  #list of tokenize word for 
  pattern_words= doc[0]
  #lemmatize each word- create base word,
  # in attemp to present related word
#   pattern_words=[lemmatize.lemmatize(word.lower()) for word in pattern_words]

  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    output_row=list(output_array)
    output_row[classes.index(doc[1])]=1
    training.append([bag,output_row])

random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")




Training data created


In [20]:
#build the model
# create model with 3 layers. First layer 128 neurons,
#second layer 64 neurons and the output contain the number of neurons
#equal the number of classes 
model= Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),), activation= 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated 
# gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=50, batch_size=5, verbose=1)
# model.save('/content/drive/My Drive/Python code/Chatbot/weather bot/weather_chatbot_model.h5', hist) # cos pyvi va stopwords
model.save('/content/drive/My Drive/Python code/Chatbot/weather bot/weather_chatbot_model2.h5', hist) # ko pyvi va stopword
print("model created")

Epoch 1/50
27840/27840 [==============================] - 19s 699us/step - loss: 0.1854 - accuracy: 0.9329
Epoch 2/50
27840/27840 [==============================] - 19s 690us/step - loss: 0.0691 - accuracy: 0.9677
Epoch 3/50
27840/27840 [==============================] - 19s 674us/step - loss: 0.0630 - accuracy: 0.9686
Epoch 4/50
27840/27840 [==============================] - 20s 702us/step - loss: 0.0613 - accuracy: 0.9686
Epoch 5/50
27840/27840 [==============================] - 19s 675us/step - loss: 0.0598 - accuracy: 0.9687
Epoch 6/50
27840/27840 [==============================] - 20s 702us/step - loss: 0.0600 - accuracy: 0.9677
Epoch 7/50
27840/27840 [==============================] - 19s 678us/step - loss: 0.0609 - accuracy: 0.9672
Epoch 8/50
27840/27840 [==============================] - 19s 679us/step - loss: 0.0587 - accuracy: 0.9679
Epoch 9/50
27840/27840 [==============================] - 19s 693us/step - loss: 0.0586 - accuracy: 0.9676
Epoch 10/50
27840/27840 [============

In [0]:
print(len(train_x))
print(len(train_y))
# print(train_x[:20])
print(len(train_x[0]))
print(train_y[:5])

1482
1482
57
[[0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0]]


In [0]:
!pip install fuzzywuzzy

In [0]:
from fuzzywuzzy import fuzz
Str1 = "Apple Inc."
Str2 = "apple Inc"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
print(Ratio)

95


/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [0]:
with open("/content/drive/My Drive/Python code/Chatbot/data.txt") as f:
    city=f.read().splitlines()
    city= [ci.lower() for ci in city]

# print(city)


a="ngày mai thời tiết La gi như thế nào"

rate= [fuzz.ratio(a)]


#Mạng CNN

In [31]:

from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from keras.utils import np_utils
from pyvi import ViTokenizer, ViPosTagger


# Define CNN architecture
embedding_dim = 128
model2    = Sequential()
model2.add(layers.Embedding(len(train_x), embedding_dim, input_length=len(train_x[0])))
model2.add(layers.Conv1D(128, 5, activation='relu'))
model2.add(layers.GlobalMaxPooling1D())
model2.add(layers.Dense(128, activation='relu'))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(len(train_y[0]), activation='softmax'))
model2.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
        
#fitting and saving the model 
hist = model2.fit(np.array(train_x), np.array(train_y), epochs=30, batch_size=5, verbose=1)
# model.save('/content/drive/My Drive/Python code/Chatbot/weather bot/weather_chatbot_model.h5', hist) # cos pyvi va stopwords
model2.save('/content/drive/My Drive/Python code/Chatbot/weather bot/weather_chatbot_model3.h5', hist) # ko pyvi va stopword
print("model created")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/30
11712/11712 [==============================] - 13s 1ms/step - loss: 1.1657 - accuracy: 0.5346
Epoch 2/30
11712/11712 [==============================] - 13s 1ms/step - loss: 0.4696 - accuracy: 0.7941
Epoch 3/30
11712/11712 [==============================] - 13s 1ms/step - loss: 0.3546 - accuracy: 0.8410
Epoch 4/30
11712/11712 [==============================] - 13s 1ms/step - loss: 0.3123 - accuracy: 0.8538
Epoch 5/30
11712/11712 [==============================] - 13s 1ms/step - loss: 0.2867 - accuracy: 0.8648
Epoch 6/30
11712/11712 [==============================] - 13s 1ms/step - loss: 0.2669 - accuracy: 0.8727
Epoch 7/30
11712/11712 [==============================] - 13s 1ms/step - loss: 0.2639 - accuracy: 0.8741
Epoch 8/30
11712/11712 [==============================] - 13s 1ms/step - loss: 0.2514 - accuracy: 0.8756
Epoch 9/30
11712/11712 [==============================] - 13s 1ms/step - loss: 0.2468 - accuracy: 0.8816
Epoch 10/30
11712/11712 [==============================

In [56]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model_real = load_model('/content/drive/My Drive/Python code/Chatbot/weather bot/model_no_pyvi/weather_chatbot_model2.h5')
import json
# import random
# print(open('intents.json').read()[440:446])
# intents = json.loads(open('intents.json').read())
intents= json.loads(open('/content/drive/My Drive/Python code/Chatbot/weather bot/weather_intents.json',"rb").read().decode("utf-8", "ignore"))
words = pickle.load(open('/content/drive/My Drive/Python code/Chatbot/weather bot/model_no_pyvi/weather_words2.pkl','rb'))
classes = pickle.load(open('/content/drive/My Drive/Python code/Chatbot/weather bot/model_no_pyvi/weather_classes2.pkl','rb'))

with open('/content/drive/My Drive/Python code/Chatbot/weather bot/stopwords.txt') as f:
    stopwords_1= f.read().splitlines()

def clean_up_sentence(sentence):
    
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # sentence_words = ViTokenizer.tokenize(sentence)
    # stem each word - create short form for word
    # sentence_words = [word.lower() for word in sentence_words if word not in stopwords_1 ]
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words ]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# def getResponse(ints, intents_json):
#     tag = ints[0]['intent']
#     list_of_intents = intents_json['intents']
#     for i in list_of_intents:
#         if(i['tag']== tag):
#             result = random.choice(i['responses'])
#             break
#     return result


# def chatbot_response(text):
#     ints = predict_class(text, model)
#     res = getResponse(ints, intents)
#     return res

a="mai rán trứng không cần mỡ được không"
result=predict_class(a,model_real)
for re in result:
    print(re["intent"] ," : ", re["probability"])


precipitation  :  1.0
